In [1]:
import os
import json
import numpy as np
from openai import OpenAI
import traceback
from dotenv import load_dotenv
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd


# .env 파일 로드
load_dotenv('.env')

# API_KEY 값을 가져옴
openai_api_key = os.getenv('OPENAI_API_KEY')



os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정

client_gpt = OpenAI()

In [ ]:
datas = pd.read_csv('stock_dataset.csv', encoding='utf-8')

# 0번부터 1000번까지의 행만 선택
datas = datas.iloc[0: 5]


   id stock_id                                         input_text answer
0   1  STOCK_1  다음 표는 주식 STOCK_1에 대한 분석 결과입니다. 표를 바탕으로 향후 STOC...   [증가]
1   2  STOCK_2  다음 표는 주식 STOCK_2에 대한 분석 결과입니다. 표를 바탕으로 향후 STOC...   [감소]
2   3  STOCK_3  다음 표는 주식 STOCK_3에 대한 분석 결과입니다. 표를 바탕으로 향후 STOC...   [증가]
3   4  STOCK_4  다음 표는 주식 STOCK_4에 대한 분석 결과입니다. 표를 바탕으로 향후 STOC...   [감소]
4   5  STOCK_5  다음 표는 주식 STOCK_5에 대한 분석 결과입니다. 표를 바탕으로 향후 STOC...   [증가]


In [65]:
def generate_explanation_from_stock(data):
    """
    Generates explanation for stock data using GPT-4o-mini model.
    """
    # Extract context fields

    # You are a skilled question generator focusing on news content. Using the provided title, dates, and content, generate exactly 5 high-quality question-answer pairs in Korean. Each question should focus on expanding the reader's understanding of the news context. Each question should align with the following categories to capture important aspects of news content: summary, main points, specific details, real-world implications, and broader significance.

    # Define prompt to ensure clear QA pair generation around the keyword and context
    
    stock = data['input_text']
    
    
    prompt = f"""
    You are a skilled stock market analyst. Using the provided stock data, analyze and determine whether the expected trend is an increase ([증가]) or a decrease ([감소]). Generate a detailed explanation in Korean based on your analysis, and specify whether the trend indicates an expected increase or decrease. Your explanation should cite specific data points to support your conclusion.

    If any data points suggest a deviation from the inferred trend, acknowledge these points and provide reasons why they might represent short-term fluctuations, or offer context as to why they may not impact the overall trend.

    The stock data is as follows:
    {stock}

    ### Explanation:
    - Start by analyzing key indicators such as closing prices, adjusted closing prices, and incremental values (e.g., `inc-5`, `inc-10`).
    - Determine and state the inferred trend ([증가] or [감소]) based on the data.
    - Provide specific data points that support this trend.
    - Acknowledge any short-term deviations and justify why they may not affect the inferred trend.

    Please respond in Korean, providing a comprehensive and balanced explanation of the expected trend and supporting rationale.
    
    ### JSON Output:
    Respond in the following JSON format:
    json
    {{
    "expectation": "<[증가] or [감소], based on your analysis>",
    "explanation": "<Detailed explanation in Korean, based on the stock data, supporting your chosen expectation>"
    }}
    """
    
    response = client_gpt.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{'role': 'user', 'content': prompt}],
        response_format = { 'type': "json_object" }
    )
    
    # Process the response
    qa_output = response.choices[0].message.content
    
    return qa_output


In [ ]:
results = []
for i in range(len(datas)):
    data = datas.iloc[i]
    result = generate_explanation_from_stock(data)
    result = json.loads(result)
    
    if result['expectation'] == data['answer']:
        results.append(result)


    print('설명: ' + result['explanation'])
    print('실제값: ' + data['answer'])
    print('예측값: ' + result['expectation'])

설명: STOCK_1의 주가 분석 결과, 전반적으로 상승하는 추세를 보이고 있습니다. 특히, 'close' 및 'adj-close' 가격이 최근 몇 번의 거래일에서 증가하는 모습을 확인할 수 있습니다. 예를 들어, 2019년 3월 15일에는 주가가 1.2로 마감되었고, 이후 3월 18일에는 2.8로 급상승하였으며, 3월 19일에도 2.9로 다시 올랐습니다. 이는 연속적인 상승을 나타냅니다.

또한, 'inc-5', 'inc-10' 등의 증가폭을 살펴보면, 5일 전과 10일 전과 비교했을 때 주가가 각각 8.4, 10.2로 나타나, 현재 주가가 개선되고 있는 지표를 보이고 있습니다. 특히 3월 18일부터 20일까지는 주가가 과거 데이터에 비추어 볼 때 굉장히 긍정적인 변화를 보여줍니다.

또한, 3월 14일에는 약간의 조정이 있었지만, 이는 단기적인 변동으로 해석될 수 있으며 전체적인 상승세에는 큰 영향을 미치지 않을 것으로 보입니다. 특히 이전의 데이터에서 볼 수 있는 큰 상승폭을 고려했을 때, 주가의 지속적인 상승 추세가 예상됩니다.

결론적으로, STOCK_1의 주가는 앞으로 [증가]할 것으로 기대됩니다. 최근의 회복세, 꾸준한 상승 데이터, 그리고 단기적인 조정은 전체적인 상승 트렌드에 큰 걸림돌이 되지 않을 것입니다.
실제값: [증가]
예측값: [증가]
설명: 주식 STOCK_2의 분석 결과를 살펴보면, 전반적으로 하락세에 있는 것으로 보입니다. 여러 지표들 중에서 특히 주가의 종가와 조정 종가(adjusted close)에서 하락하는 경향이 나타납니다. 예를 들어, 2022년 4월 27일의 종가는 -1.3으로 시작하여, 5월 10일에는 -2.2로 감소했습니다. 이 기간 동안 주가는 지속적으로 음의 값을 기록하며 하락세를 보이고 있습니다.

또한 inc-5, inc-10, inc-15와 같은 증가 값들도 대부분 음수를 기록하고 있어, 단기적인 거래에서 주가 상승을 기대하기가 어렵습니다. 특히 5월 9일에 -7.2, 5월 10일에 -8.1의 감소폭은 주가

[{'expectation': '[증가]',
  'explanation': "STOCK_1의 주가 분석 결과, 전반적으로 상승하는 추세를 보이고 있습니다. 특히, 'close' 및 'adj-close' 가격이 최근 몇 번의 거래일에서 증가하는 모습을 확인할 수 있습니다. 예를 들어, 2019년 3월 15일에는 주가가 1.2로 마감되었고, 이후 3월 18일에는 2.8로 급상승하였으며, 3월 19일에도 2.9로 다시 올랐습니다. 이는 연속적인 상승을 나타냅니다.\n\n또한, 'inc-5', 'inc-10' 등의 증가폭을 살펴보면, 5일 전과 10일 전과 비교했을 때 주가가 각각 8.4, 10.2로 나타나, 현재 주가가 개선되고 있는 지표를 보이고 있습니다. 특히 3월 18일부터 20일까지는 주가가 과거 데이터에 비추어 볼 때 굉장히 긍정적인 변화를 보여줍니다.\n\n또한, 3월 14일에는 약간의 조정이 있었지만, 이는 단기적인 변동으로 해석될 수 있으며 전체적인 상승세에는 큰 영향을 미치지 않을 것으로 보입니다. 특히 이전의 데이터에서 볼 수 있는 큰 상승폭을 고려했을 때, 주가의 지속적인 상승 추세가 예상됩니다.\n\n결론적으로, STOCK_1의 주가는 앞으로 [증가]할 것으로 기대됩니다. 최근의 회복세, 꾸준한 상승 데이터, 그리고 단기적인 조정은 전체적인 상승 트렌드에 큰 걸림돌이 되지 않을 것입니다."},
 {'expectation': '[감소]',
  'explanation': '주식 STOCK_2의 분석 결과를 살펴보면, 전반적으로 하락세에 있는 것으로 보입니다. 여러 지표들 중에서 특히 주가의 종가와 조정 종가(adjusted close)에서 하락하는 경향이 나타납니다. 예를 들어, 2022년 4월 27일의 종가는 -1.3으로 시작하여, 5월 10일에는 -2.2로 감소했습니다. 이 기간 동안 주가는 지속적으로 음의 값을 기록하며 하락세를 보이고 있습니다.\n\n또한 inc-5, inc-10, inc-15와 같은 증가 값들도 대부분 음수를 기록하고 있어